In [2]:
import os
import urllib

import numpy as np
import tensorflow as tf
import pandas as pd


IRIS_TRAINING = "../data/iris/iris_training.csv"
IRIS_TRAINING_URL = "http://download.tensorflow.org/data/iris_training.csv"

IRIS_TEST = "../data/iris/iris_test.csv"
IRIS_TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"

CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth',
                    'PetalLength', 'PetalWidth', 'Species']

SPECIES = ['Setosa', 'Versicolor', 'Virginica']

BATCH_SIZE=10




/home/ubuntu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:

def load_data(y_name='Species'):
    """Returns the iris dataset as (train_x, train_y), (test_x, test_y)."""
   
    train = pd.read_csv(IRIS_TRAINING, names=CSV_COLUMN_NAMES, header=0)
    train_x, train_y = train, train.pop(y_name)

    test = pd.read_csv(IRIS_TEST, names=CSV_COLUMN_NAMES, header=0)
    test_x, test_y = test, test.pop(y_name)

    return (train_x, train_y, test_x, test_y)

train_x, train_y, test_x, test_y = load_data()

In [4]:
# Specify that all features have real-value data
feature_columns = []
for key in train_x.keys():
    feature_columns.append(tf.feature_column.numeric_column(key=key))

In [6]:

# Build 3 layer DNN with 10, 20, 10 units respectively.
classifier = tf.estimator.DNNClassifier(feature_columns=feature_columns,
                                            hidden_units=[10, 20, 10],
                                            n_classes=3,
                                            model_dir="/tmp/iris_model")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/iris_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fea7a22b780>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [7]:
# Define the input function

def train_input_fn(features, labels, batch_size):
    """An input function for training"""
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle, repeat, and batch the examples.
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)

    # Return the dataset.
    return dataset

# Fit model.
# Train the Model.
classifier.train(
    input_fn=lambda:train_input_fn(train_x, train_y, BATCH_SIZE), steps=1000)


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/iris_model/model.ckpt-1
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 2 into /tmp/iris_model/model.ckpt.
INFO:tensorflow:loss = 10.38555, step = 2
INFO:tensorflow:global_step/sec: 721.496
INFO:tensorflow:loss = 7.1726027, step = 102 (0.140 sec)
INFO:tensorflow:global_step/sec: 1074
INFO:tensorflow:loss = 6.0283756, step = 202 (0.093 sec)
INFO:tensorflow:global_step/sec: 1018.73
INFO:tensorflow:loss = 3.3351402, step = 302 (0.098 sec)
INFO:tensorflow:global_step/sec: 1029.5
INFO:tensorflow:loss = 2.6257076, step = 402 (0.097 sec)
INFO:tensorflow:global_step/sec: 970.046
INFO:tensorflow:loss = 2.2845247, step = 502 (0.103 sec)
INFO:tensorflow:global_step/sec: 979.405
INFO:tensorflow:loss = 1.1301894, step = 602 (0.1

In [16]:

def eval_input_fn(features, labels, batch_size):
    """An input function for evaluation or prediction"""
    
    features=dict(features)
    inputs = features if labels is None else (features, labels)
   
    # Convert the inputs to a Dataset.
    return tf.data.Dataset.from_tensor_slices(inputs).batch(batch_size)


# Evaluate the model.
eval_result = classifier.evaluate(
    input_fn=lambda:eval_input_fn(test_x, test_y, BATCH_SIZE))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-06-19-22:40:31
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/iris_model/model.ckpt-1001
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-06-19-22:40:32
INFO:tensorflow:Saving dict for global step 1001: accuracy = 0.96666664, average_loss = 0.13056086, global_step = 1001, loss = 1.3056086

Test set accuracy: 0.967



In [17]:
# Generate predictions from the model
expected = ['Setosa', 'Versicolor', 'Virginica']
predict_x = {
    'SepalLength': [5.1, 5.9, 6.9],
    'SepalWidth': [3.3, 3.0, 3.1],
    'PetalLength': [1.7, 4.2, 5.4],
    'PetalWidth': [0.5, 1.5, 2.1],
}

predictions = classifier.predict(
    input_fn=lambda:iris_data.eval_input_fn(predict_x,
                                            batch_size=args.batch_size))

pr